In [ ]:
# default_exp model.core

# Model

> API details. @nathan

In [ ]:
# export
import tensorflow as tf

from abc import ABC, abstractmethod
from pathlib import Path

In [ ]:
# export
class Model(ABC):
    # TODO: Add generating the model config (but only the pieces we care about, i.e., the num layers, heads, dim size, emb size, etc) so that we can easily save it to a file for organizing evaluation results
    def __init__(self, tokenizer, model):
        self.tokenizer = tokenizer
        self.model = model

    @abstractmethod
    def tokenize(self, method):
        pass

    @abstractmethod
    def train(self, ds, epochs):
        pass

    @abstractmethod
    def get_probs(self, inputs):
        pass

    # TODO: Add save method that handles saving model and tokenizer to disc

In [ ]:
# export

# Tensorflow Huggingface Transformer
class TransformerModel(Model):
    def tokenize(self, method):
        return self.tokenizer(method, return_tensors="tf")

    def get_probs(self, inputs):
        outputs = self.model(inputs)
        logits = outputs[0]
        probs = tf.nn.softmax(logits)

        return probs


# Tensorflow GRU Model
# class GRUModel(Model):
#     def tokenize(self, method):
#         pass

#     def get_probs(self, method):
#         pass

In [ ]:
# export
class RNNModel(Model):
    _RNN_TYPE = {
        "rnn": tf.keras.layers.SimpleRNN,
        "gru": tf.keras.layers.GRU,
        "lstm": tf.keras.layers.LSTM,
    }

    def __init__(
        self,
        rnn_type,
        n_layers,
        vocab_size,
        embedding_dim,
        rnn_units,
        batch_size,
        out_path,
        tokenizer,
    ):
        self.config_name = (
            f"{rnn_type}_vocab{vocab_size}_embed{embedding_dim}_units{rnn_units}"
        )
        self.out_path = Path(out_path) / self.config_name
        self.callbacks = [
            tf.keras.callbacks.ModelCheckpoint(
                filepath=self.out_path / "ckpt_{epoch}", save_weights_only=True
            )
        ]

        self.tokenizer = tokenizer
        layer = RNNModel._RNN_TYPE[rnn_type]
        rnn_layers = [
            layer(
                rnn_units,
                return_sequences=True,
                # I think we need to have this not be stateful since we don't
                # chop up examples
                # stateful=True,
                recurrent_initializer="glorot_uniform",
                # following BigCode != Big Vocab Paper
                dropout=0.1,
            )
            for _ in range(n_layers)
        ]
        self.model = tf.keras.Sequential(
            [
                tf.keras.layers.Embedding(
                    input_dim=vocab_size,
                    output_dim=embedding_dim,
                    mask_zero=True,  # Zero cannot be used in the vocabulary
                    batch_input_shape=[batch_size, None],
                ),
            ]
            + rnn_layers
            + [
                tf.keras.layers.Dense(vocab_size),
            ]
        )

        super().__init_(self.tokenizer, self.model)

    # TODO add code to easily train model
    def train(self, dataset, epochs):
        self.model.compile(optimizer="adam", loss=_loss)
        _ = self.model.fit(dataset, epochs=epochs, callbacks=self.callbacks)

    def generate(self, n, temperature=1.0):
        # Evaluation step (generating text using the learned model)

        # Converting our start string to numbers (vectorizing)
        input_eval = [self.tokenizer.bos_token_id]
        input_eval = tf.expand_dims(input_eval, 0)

        # Empty string to store our results
        text_generated = []

        # Here batch size == 1
        self.model.reset_states()
        for i in range(n):
            predictions = self.model(input_eval)
            # remove the batch dimension
            predictions = tf.squeeze(predictions, 0)

            # using a categorical distribution to predict the character returned by the model
            predictions = predictions / temperature
            predicted_id = tf.random.categorical(predictions, num_samples=1)[
                -1, 0
            ].numpy()

            # Pass the predicted character as the next input to the model
            # along with the previous hidden state
            input_eval = tf.expand_dims([predicted_id], 0)

            text_generated.append(predicted_id)

        return self.tokenizer.decode(text_generated), text_generated

    def get_probs(self, method):
        pass

In [ ]:
# hide
from nbdev.export import notebook2script

notebook2script()